# Happy Hours in Downtown Cincinnati

In [5]:
## all imports
from IPython.display import HTML
import numpy as np
import urllib2
import bs4 #this is beautiful soup
import time
import operator
import socket
import cPickle
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

from secret import *

Beautiful Soup is a Python library for pulling data out of HTML and XML files. It works with parsers to provide idiomatic ways of navigating, searching, and modifying the parse tree. It saves hours or days of work going through the HTML tag.

For looking at bars offering happy hours, I used Downtown Cincinnati website. You can look at the website to get a feel of how the happy hour info is listed

In [13]:
url = 'http://www.downtowncincinnati.com/exploring-downtown/happy-hours'
source = urllib2.urlopen(url).read()


In [15]:
# get links of the bars offering happy hour to extract information

## get bs4 object
soup = bs4.BeautifulSoup(source)

## compare the two print statements
#print soup
#print soup.prettify()

## get attribute `href`
link_list = [l.get('href') for l in soup.findAll('a') ]
link_list = [l for l in link_list if '/exploring-downtown/establishment/' in l]
link_list

['/exploring-downtown/establishment/a-tavola-pizza',
 '/exploring-downtown/establishment/bistro-on-elm',
 '/exploring-downtown/establishment/boi-na-braza',
 '/exploring-downtown/establishment/che',
 '/exploring-downtown/establishment/neons-unplugged',
 '/exploring-downtown/establishment/holy-grail-tavern-grille',
 '/exploring-downtown/establishment/igbys',
 '/exploring-downtown/establishment/japps-since-1879',
 '/exploring-downtown/establishment/jimmy-buffetts-margaritaville-restaurant',
 '/exploring-downtown/establishment/kaze',
 '/exploring-downtown/establishment/lackman-the',
 '/exploring-downtown/establishment/madonnas-bar-grill',
 '/exploring-downtown/establishment/mccormick-schmicks-seafood-restaurant',
 '/exploring-downtown/establishment/miltons',
 '/exploring-downtown/establishment/mortons-the-steakhouse---new-look',
 '/exploring-downtown/establishment/nada',
 '/exploring-downtown/establishment/neuf',
 '/exploring-downtown/establishment/omalleys-in-the-alley',
 '/exploring-down

In [9]:
# Get the bar details
est_name = []
est_desc = []
est_address =[]
est_hours = []
est_happyhr = []
est_link = []
est_phnum =[]


for link in link_list:
    url = 'http://www.downtowncincinnati.com' + link
    source = urllib2.urlopen(url).read()
    soup = bs4.BeautifulSoup(source)
    est_details = soup.find(id="establishment_details")
    est_name.append([str(t.get_text().encode('utf-8')) for t in est_details.findAll('h3')][0])
    est_info = [str(t.get_text().encode('utf-8')) for t in est_details.findAll('p')]
    est_desc.append(est_info[0])
    est_address.append(est_info[1])
    est_hours.append(est_info[2])
    if len(est_info) > 3:
        est_happyhr.append(est_info[3])
    else:
        est_happyhr.append('NA')
    est_link.append([l.get('href') for l in est_details.findAll('a')][0])
    est_phnum.append([str(t.get_text().encode('utf-8')) for t in est_details.findAll('h4')][0])
      

In [11]:
# Create a pandas dataframe

DTCincyRest = pd.DataFrame({
        "Name": est_name, 
        "Short Description": est_desc, 
        "Address": est_address, 
        "Hours":est_hours,
        "Happy Hour":est_happyhr,
        "Website":est_link,
        "Phone Number":est_phnum
    })

DTCincyRest = DTCincyRest.iloc[:,[3,5,2,1,0,4,6]]
DTCincyRest.head()

,Name,Short Description,Hours,Happy Hour,Address,Phone Number,Website
0,A Tavola Pizza,Neapolitan pizza and other Italian cuisine mad...,"Mon-Thu, 11am-11am; Fri & Sat, 11am-2am; Sun, ...",Tue-Fri 4-6 p.m.,1220 Vine St.,(513) 246-0192,http://www.atavolapizza.com/
1,Bistro on Elm,"Restaurant offers breakfast, lunch or dinner. ...","Mon-Fri, 11am-10pm; Sat- Sun,12pm-10pm, Bar 12...","Mon-Fri, 4-7, half price apps, $1 off draft beers",150 W. Fifth St.,(513) 352-2189,/exploring-downtown/establishment/bakes-place
2,Boi Na Braza,Brazilian steak house (a churrascaria). 15 cut...,"Mon- Sat, 5pm-9:45pm; Sun, 5pm-8:45pm","Mon-Fri, 4:30pm-7pm, half price well drinks an...",441 Vine St.,(513) 421-7111,http://www.boinabraza.com/
3,Che,Serves a combination of Italian and Latin Amer...,"Sun - Thurs, 11am-1am; Fri & Sat, 11am-2am",NA,1342 Walnut St.,(513) 978-1706,http://www.checincinnati.com/menu.html
4,Famous Neons Unplugged,"Known for its beautiful courtyard bar, complet...","Mon-Fri, 4pm-2am; Sat, 3pm-2am; Sun, 12pm-12am",M-F 4-8,208 E Twelfth St.,(513) 479-1030,http://www.neons-unplugged.com/


In [ ]:
# Export it to a CSV
DTCincyRest.to_csv('HappyHour.csv',index = False)